# Multi-Objective Algorithm

In this notebook, we will use the Reference Vector Guided Evolutionary Algorithm (RVEA) to find the optimal solutions of the DTLZ2 problem.

In [ ]:
# install evox, skip it if you have already installed evox
try:
    import evox
except ImportError:
    !pip install --disable-pip-version-check --upgrade -q evox

In [1]:
import time

import torch
from torch.profiler import ProfilerActivity, profile

from evox.algorithms import RVEA
from evox.core import jit, use_state
from evox.metrics import igd
from evox.problems.numerical import DTLZ2
from evox.workflows import StdWorkflow

In [2]:
# Use GPU first to run the code.
torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.get_default_device())

cuda:0


In [3]:
# Init the problem, algorithm and workflow.
prob = DTLZ2(m=3)
pf = prob.pf()
algo = RVEA(pop_size=100, n_objs=3, lb=-torch.zeros(12), ub=torch.ones(12))
workflow = StdWorkflow()
workflow.setup(algo, prob)
workflow.init_step()

# Init the state_step and the jit_state_step
state_step = use_state(lambda: workflow.step)
state = state_step.init_state()
jit_state_step = jit(state_step, trace=True, example_inputs=(state,))

In [4]:
# Run the workflow for 100 steps
t = time.time()
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    for i in range(100):
        state = jit_state_step(state)
        fit = state["self.algorithm.fit"]
        fit = fit[~torch.isnan(fit).any(dim=1)]
        print(igd(fit, pf))

tensor(0.4494, device='cuda:0')
tensor(0.3799, device='cuda:0')
tensor(0.3475, device='cuda:0')
tensor(0.3060, device='cuda:0')
tensor(0.2827, device='cuda:0')
tensor(0.2448, device='cuda:0')
tensor(0.2247, device='cuda:0')
tensor(0.2025, device='cuda:0')
tensor(0.1823, device='cuda:0')
tensor(0.1693, device='cuda:0')
tensor(0.1469, device='cuda:0')
tensor(0.1367, device='cuda:0')
tensor(0.1238, device='cuda:0')
tensor(0.1166, device='cuda:0')
tensor(0.1094, device='cuda:0')
tensor(0.1029, device='cuda:0')
tensor(0.0985, device='cuda:0')
tensor(0.0917, device='cuda:0')
tensor(0.0872, device='cuda:0')
tensor(0.0833, device='cuda:0')
tensor(0.0816, device='cuda:0')
tensor(0.0770, device='cuda:0')
tensor(0.0747, device='cuda:0')
tensor(0.0730, device='cuda:0')
tensor(0.0700, device='cuda:0')
tensor(0.0686, device='cuda:0')
tensor(0.0671, device='cuda:0')
tensor(0.0651, device='cuda:0')
tensor(0.0640, device='cuda:0')
tensor(0.0631, device='cuda:0')
tensor(0.0624, device='cuda:0')
tensor(0

In [5]:
print(prof.key_averages().table())
torch.cuda.synchronize()
print(time.time() - t)

--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                 aten::equal         1.32%       8.399ms         6.45%      40.980ms      76.741us       2.226ms         0.27%      80.683ms     151.092us           0 b         -32 b           0 b    -154.00 Kb           534  
                          